# Data Cleaning
When collecting data from online or real-time sources, the dataset is always a bit dirty. There may be missing values, nulls, or just incorrect inputs. 

Our columns are relatively clean, so we won't show a very expansive set of cleaning tools, but feel free to check out more of our workshops to experiment with other types of data.

In [36]:
import os
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings('ignore')

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

Current working directory: /Users/cory/Documents/git/edge-failure-prediction/notebooks


In [37]:
# check if the a directory exists, if not create it
outdir = './scratch'

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [38]:
# creates a connection to a database
conn = psycopg2.connect(
database="predict-db", 
    user='predict-db', 
    password='failureislame', 
    host='localhost')

GET_ALL_ROWS = 'Select * from waterpump order by timestamp'

try:
    with conn:
        #Pull our dataset into a pandas dataframe
        df = pd.read_sql_query(GET_ALL_ROWS, conn)
        df.set_index('timestamp', inplace=True)
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
finally:
    conn.close()

### Lets make a copy of the dataset, so that if we make a mistake or just want a clean version of the dataset, we don't need to run that cell above again.

In [39]:
df_original =  df.copy()

### As we said before, we have some nulls in the data. Let's see if any columns are unusable.

In [40]:
nulls_series = df.isnull().sum()
print(nulls_series.sort_values())

machine_status         0
sensor_19             16
sensor_23             16
sensor_24             16
sensor_27             16
sensor_28             16
sensor_31             16
sensor_20             16
sensor_33             16
sensor_21             16
sensor_35             16
sensor_36             16
sensor_37             16
sensor_34             16
sensor_11             19
sensor_12             19
sensor_13             19
sensor_05             19
sensor_04             19
sensor_03             19
sensor_02             19
sensor_10             19
sensor_26             20
sensor_14             21
sensor_47             27
sensor_41             27
sensor_40             27
sensor_39             27
sensor_38             27
sensor_44             27
sensor_45             27
sensor_46             27
sensor_42             27
sensor_43             27
sensor_48             27
sensor_49             27
sensor_16             31
sensor_25             36
sensor_22             41
sensor_17             46


### Something looks wrong with sensor_15 data...

In [41]:
df['sensor_15'].unique()

array([None], dtype=object)

In [42]:
# drop it like it's hot
df.drop('sensor_15', axis=1, errors='ignore', inplace=True)

In [43]:
# select the number of columns with too many null values
number_removed = 3
empty_cols = nulls_series.sort_values().tail(number_removed)
display(empty_cols)

# get the names of the columns in a list
bad_col_list = list(empty_cols.keys())

# drop the bad columns
df.drop(bad_col_list, axis=1, errors='ignore', inplace=True)
print(df.columns)

sensor_51     15383
sensor_50     77017
sensor_15    220320
dtype: int64

Index(['sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
       'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
       'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14',
       'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
       'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
       'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
       'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
       'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
       'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
       'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'machine_status'],
      dtype='object')


### When we ultimately train a model, we'll need to get all of or columns into numbers
### If a non-numerical feature has a discrete distribution, we can implement a practice called one-hot-encoding that will assign our values 0 (False) or 1 (True)

In [44]:
# we have an in-between stage, 'recovering', so we'll label it 0.5

# a dictionary can be used to one-to-one map values in a series
status_map = {'NORMAL': 0, 'BROKEN': 1, 'RECOVERING':0.5}

df['machine_status'] = df['machine_status'].map(status_map)

### The index of our dataframe, the time, contains strings. Let's give them a smarter type that understands time.

In [45]:
df.index = pd.to_datetime(df.index)

### Now that all of our columns are numerical, we can run some math operations ourselves for testing purposes.

In [46]:
df.describe().iloc[:, :15]

,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,sensor_13,sensor_14
count,210112.000000,219951.000000,220301.000000,220301.000000,220301.000000,220301.000000,215522.000000,214869.000000,215213.000000,215725.000000,220301.000000,220301.000000,220301.000000,220301.000000,220299.000000
mean,2.372221,47.591611,50.867392,43.752481,590.673936,73.396414,13.501537,15.843152,15.200721,14.799210,41.470339,41.918319,29.136975,7.078858,376.860041
std,0.412227,3.296666,3.666820,2.418887,144.023912,17.298247,2.163736,2.201155,2.037390,2.091963,12.093519,13.056425,10.113935,6.901755,113.206382
min,0.000000,0.000000,33.159720,31.640620,2.798032,0.000000,0.014468,0.000000,0.028935,0.000000,0.000000,0.000000,0.000000,0.000000,32.409550
25%,2.438831,46.310760,50.390620,42.838539,626.620400,69.976260,13.346350,15.907120,15.183740,15.053530,40.705260,38.856420,28.686810,1.538516,418.103250
50%,2.456539,48.133678,51.649300,44.227428,632.638916,75.576790,13.642940,16.167530,15.494790,15.082470,44.291340,45.363140,32.515830,2.929809,420.106200
75%,2.499826,49.479160,52.777770,45.312500,637.615723,80.912150,14.539930,16.427950,15.697340,15.118630,47.463760,49.656540,34.939730,12.859520,420.997100
max,2.549016,56.727430,56.032990,48.220490,800.000000,99.999880,22.251160,23.596640,24.348960,25.000000,76.106860,60.000000,45.000000,31.187550,500.000000


### Let's check all the means of our sensors. And while we're at it, let's fill in any null values with those means, so we don't change the average.

In [47]:
col_averages = df.mean()
print(col_averages)
df.fillna(value=col_averages, inplace=True)

sensor_00           2.372221
sensor_01          47.591611
sensor_02          50.867392
sensor_03          43.752481
sensor_04         590.673936
sensor_05          73.396414
sensor_06          13.501537
sensor_07          15.843152
sensor_08          15.200721
sensor_09          14.799210
sensor_10          41.470339
sensor_11          41.918319
sensor_12          29.136975
sensor_13           7.078858
sensor_14         376.860041
sensor_16         416.472892
sensor_17         421.127517
sensor_18           2.303785
sensor_19         590.829775
sensor_20         360.805165
sensor_21         796.225942
sensor_22         459.792815
sensor_23         922.609264
sensor_24         556.235397
sensor_25         649.144799
sensor_26         786.411781
sensor_27         501.506589
sensor_28         851.690339
sensor_29         576.195305
sensor_30         614.596442
sensor_31         863.323100
sensor_32         804.283915
sensor_33         486.405980
sensor_34         234.971776
sensor_35     

### We should be good to go into further analysis, let's save a csv file so our next notebook can access our updated data.

In [48]:
df.to_csv(outdir + '/clean-df.csv')